<a href="https://colab.research.google.com/github/detayans/Bagging-and-Boosting-Project/blob/main/ProjectBagging%26Boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step I : Load the Data

In [102]:
# Step 1 - Load Data
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
%matplotlib inline
dataset = pd.read_csv("/content/talking_data.csv")

In [75]:
dataset

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0
...,...,...,...,...,...,...,...,...
99995,124883,11,1,19,122,2017-11-09 13:25:41,NaN,0
99996,85150,9,1,13,244,2017-11-07 11:25:43,NaN,0
99997,18839,3,1,13,19,2017-11-08 11:38:42,NaN,0
99998,114276,15,1,12,245,2017-11-08 17:55:21,NaN,0


In [76]:
dataset.dtypes

ip                  int64
app                 int64
device              int64
os                  int64
channel             int64
click_time         object
attributed_time    object
is_attributed       int64
dtype: object

# Step II : Explore and Find Unique Values

In [77]:
len(dataset.ip.unique())

34857

In [78]:
len(dataset.app.unique())

161

In [79]:
len(dataset.device.unique())

100

In [80]:
len(dataset.os.unique())

130

In [81]:
len(dataset.channel.unique())

161

In [82]:
len(dataset.click_time.unique())

80350

In [83]:
len(dataset.attributed_time.unique())

228

In [84]:
len(dataset.is_attributed.unique())

2

In [85]:
miss_val = dataset.isna().sum()
miss_val

ip                     0
app                    0
device                 0
os                     0
channel                0
click_time             0
attributed_time    99773
is_attributed          0
dtype: int64

In [86]:
dataset.shape

(100000, 8)

# Step III : Drop the attributed_time columns

In [87]:
dataset = dataset.drop(['attributed_time'], axis = 1)

In [88]:
dataset.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [89]:
dataset.shape

(100000, 7)

In [90]:
dataset.app.value_counts()

3      18279
12     13198
2      11737
9       8992
15      8595
       ...  
407        1
145        1
271        1
398        1
92         1
Name: app, Length: 161, dtype: int64

In [91]:
app_val = dataset['app'].value_counts()> 2000

In [92]:
app_val

3       True
12      True
2       True
9       True
15      True
       ...  
407    False
145    False
271    False
398    False
92     False
Name: app, Length: 161, dtype: bool

In [93]:
app_new = app_val[app_val == True]
app_new

3     True
12    True
2     True
9     True
15    True
18    True
14    True
1     True
13    True
8     True
Name: app, dtype: bool

In [94]:
app_new.value_counts()

True    10
Name: app, dtype: int64

# Step IV: Using Click_time create columns such as month, hour, day_of_week and day_of_year

In [95]:
def timefeatures(data):
  data['datetime'] = pd.to_datetime(data['click_time'])
  data['day_of_week'] = data['datetime'].dt.dayofweek
  data['day_of_year'] = data['datetime'].dt.dayofyear
  data['month'] = data['datetime'].dt.month
  data['hour'] = data['datetime'].dt.hour
  return data


In [96]:
dataset = timefeatures(dataset)

In [97]:
dataset.head()

,ip,app,device,os,channel,click_time,is_attributed,datetime,day_of_week,day_of_year,month,hour
0,87540,12,1,13,497,2017-11-07 09:30:38,0,2017-11-07 09:30:38,1,311,11,9
1,105560,25,1,17,259,2017-11-07 13:40:27,0,2017-11-07 13:40:27,1,311,11,13
2,101424,12,1,19,212,2017-11-07 18:05:24,0,2017-11-07 18:05:24,1,311,11,18
3,94584,13,1,13,477,2017-11-07 04:58:08,0,2017-11-07 04:58:08,1,311,11,4
4,68413,12,1,1,178,2017-11-09 09:00:09,0,2017-11-09 09:00:09,3,313,11,9


In [98]:
dataset = dataset.drop(['click_time','datetime'], axis = 1)

In [99]:
dataset.head()

,ip,app,device,os,channel,is_attributed,day_of_week,day_of_year,month,hour
0,87540,12,1,13,497,0,1,311,11,9
1,105560,25,1,17,259,0,1,311,11,13
2,101424,12,1,19,212,0,1,311,11,18
3,94584,13,1,13,477,0,1,311,11,4
4,68413,12,1,1,178,0,3,313,11,9


In [100]:
X = dataset.drop(['is_attributed'], axis = 1)
Y = dataset['is_attributed']

# Step V : Train Dataset


In [103]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(80000, 9)
(80000,)
(20000, 9)
(20000,)


In [104]:
X_train,X_test,Y_train,Y_test
## applying bagging classifier algorithm
from sklearn.ensemble import BaggingClassifier

bgcl = BaggingClassifier(n_estimators=10, max_samples= .7, bootstrap=True)
bgcl = bgcl.fit(X_train, Y_train)
test_pred = bgcl.predict(X_test)
bgcl_score=bgcl.score(X_test, Y_test)
print("The Bagging classifier model accuracy is {}".format(bgcl_score))
print("The confusion matrix of Bagging classifier model is")
print(metrics.confusion_matrix(Y_test,test_pred))


The Bagging classifier model accuracy is 0.9979
The confusion matrix of Bagging classifier model is
[[19942     9]
 [   33    16]]


In [105]:
Y_pred=bgcl.predict_proba(X_test)
Y_pred


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

# Step VI: AUC/ROC by using Bagging Classifier

In [106]:
# AUC/ROC for BaggingClassifier
auc=metrics.roc_auc_score(Y_test,Y_pred[:,1])
print('AUC = {} %'.format(auc*100))


AUC = 88.53169857988807 %


## Training Data with Xgboost model

In [107]:
## applying the XGBoostClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
xgbl=XGBClassifier()
xgbl.fit(X_train,Y_train)
test_pred=xgbl.predict(X_test)
xgbl_score=xgbl.score(X_test,Y_test)
print("The XGBoostclassifier model accuracy is {}".format(xgbl_score))
print("The confusion matrix of XGBoostclassifier model is")
print(metrics.confusion_matrix(Y_test,test_pred))


The XGBoostclassifier model accuracy is 0.9978
The confusion matrix of XGBoostclassifier model is
[[19949     2]
 [   42     7]]


In [108]:
Y_pred1=xgbl.predict_proba(X_test)
Y_pred1


array([[9.9966216e-01, 3.3781320e-04],
       [9.9933451e-01, 6.6549581e-04],
       [9.9871898e-01, 1.2810524e-03],
       ...,
       [9.9570459e-01, 4.2954073e-03],
       [9.9940139e-01, 5.9859117e-04],
       [9.9964434e-01, 3.5568385e-04]], dtype=float32)

# Step VII : AUC/ROC score of XGBoost Classifier

In [109]:
# AUC/ROC for BaggingClassifier
auc=metrics.roc_auc_score(Y_test,Y_pred1[:,1])
print('AUC = {} %'.format(auc*100))


AUC = 96.82134494818428 %


# Step VIII: Conclusion

In [ ]:
XGBoost gives better AUC score than Bagging Classifier